In [2]:
import torch

import whisper
from whisper.model import MultiHeadAttention


def main():
    model = whisper.load_model("tiny.en")
    model.cpu().eval()
    patch(model)

    encoder = model.encoder.cpu()
    decoder = FunctionalDecoder(model.decoder.cpu())

    x_mel = torch.randn(1, 80, 3000)
    x_tokens = torch.zeros((1, 10), dtype=torch.long).cpu()
    x_audio = encoder(x_mel).cpu()

    cache_self_attn = torch.zeros(
        (len(decoder.keys_self_attn), 1, 1, model.dims.n_text_state),
    )
    cache_cross_attn = torch.zeros(
        (len(decoder.keys_cross_attn), 1, 1, model.dims.n_audio_state),
    )

    torch.onnx.export(
        encoder,
        (x_mel,),
        "encoder.onnx",
        input_names=["mel"],
        output_names=["audio"],
        dynamic_axes={
            "mel": {0: "batch", 2: "time"},
            "audio": {0: "batch", 1: "time"},
        },
        opset_version=17,
    )

    torch.onnx.export(
        decoder,
        (x_tokens, x_audio, cache_self_attn, cache_cross_attn),
        "decoder.onnx",
        input_names=["tokens", "audio", "cache_self_attn", "cache_cross_attn"],
        output_names=["logits", "new_cache_self_attn", "new_cache_cross_attn"],
        dynamic_axes={
            # inputs
            "tokens": {0: "batch", 1: "seq"},
            "audio": {0: "batch", 1: "time"},
            "cache_self_attn": {2: "cached_seq"},
            "cache_cross_attn": {2: "cached_time"},
            # outputs
            "logits": {0: "batch", 1: "seq"},
            "new_cache_self_attn": {1: "batch", 2: "new_cached_seq"},
            "new_cache_cross_attn": {1: "batch", 2: "new_cached_time"},
        },
        opset_version=17,
    )


def patch(model):
    for block in model.decoder.blocks:
        block.attn.__class__ = FunctionalMultiHeadAttention
        block.attn.n_ctx = model.dims.n_text_ctx

        block.cross_attn.__class__ = FunctionalMultiHeadAttention
        block.cross_attn.n_ctx = model.dims.n_audio_ctx


class FunctionalDecoder(torch.nn.Module):
    def __init__(self, decoder):
        super().__init__()
        self.decoder = decoder

        self.keys_self_attn = []
        self.keys_cross_attn = []

        for block in decoder.blocks:
            self.keys_self_attn += (block.attn.key, block.attn.value)
            self.keys_cross_attn += (block.cross_attn.key, block.cross_attn.value)

    def forward(self, x, xa, cache_self_attn, cache_cross_attn):
        kv_cache = {
            **dict(zip(self.keys_self_attn, cache_self_attn)),
            **dict(zip(self.keys_cross_attn, cache_cross_attn)),
        }

        logits = self.decoder(x, xa, kv_cache=kv_cache)
        return (
            logits,
            torch.cat([kv_cache[key].unsqueeze(0) for key in self.keys_self_attn], dim=0),
            torch.cat([kv_cache[key].unsqueeze(0) for key in self.keys_cross_attn], dim=0),
        )


class FunctionalMultiHeadAttention(MultiHeadAttention):
    def forward(self, x, xa=None, mask=None, kv_cache=None):
        k, v = self._get_kv(x, xa, kv_cache)

        q = self.query(x)
        wv, qk = self.qkv_attention(q, k, v, mask)
        return self.out(wv), qk

    def _get_kv(self, x, xa=None, kv_cache=None):
        xx = x if xa is None else xa
        assert xx is not None

        if kv_cache is None:
            return self.key(xx), self.value(xx)

        key = torch.concat([kv_cache[self.key], self.key(xx).detach()], dim=1)
        key = key[:, -self.n_ctx :, :]
        kv_cache[self.key] = key

        value = torch.concat([kv_cache[self.value], self.value(xx).detach()], dim=1)
        value = value[:, -self.n_ctx :, :]
        kv_cache[self.value] = value

        return kv_cache[self.key], kv_cache[self.value]


if __name__ == "__main__":
    main()

/tmp/ipykernel_42371/4226923902.py:26: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0207 13:52:07.387000 42371 torch/onnx/_internal/exporter/_compat.py:125] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W0207 13:52:07.928000 42371 torch/onnx/_internal/exporter/_registration.py:110] torchvision is not installed. Skipping torchvision::nms
W0207 13:52:07.930000 42371 torch/onnx/_internal/exporter/_registration.py:110] torchv

[torch.onnx] Obtain model graph for `AudioEncoder([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `AudioEncoder([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


/usr/lib/python3.12/copyreg.py:99: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)
The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 17).


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 16 of general pattern rewrite rules.


/tmp/ipykernel_42371/4226923902.py:39: UserWarning: Exporting a model while it is in training mode. Please ensure that this is intended, as it may lead to different behavior during inference. Calling model.eval() before export is recommended.
  torch.onnx.export(
/tmp/ipykernel_42371/4226923902.py:39: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0207 13:52:13.737000 42371 torch/onnx/_internal/exporter/_compat.py:125] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_vers

[torch.onnx] Obtain model graph for `FunctionalDecoder([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `FunctionalDecoder([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


/usr/lib/python3.12/copyreg.py:99: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 17).


[torch.onnx] Translate the graph into ONNX... ✅
Applied 37 of general pattern rewrite rules.


corrected enhanced

In [2]:
# whisper_to_onnx.py
# Production-grade export with external KV cache (Android / ONNX Runtime)

import os
import shutil
import torch
import whisper
from whisper.model import MultiHeadAttention


OPSET = 17


def main():
    model = whisper.load_model("tiny.en").cpu().eval()
    _patch_decoder(model)

    encoder = model.encoder
    decoder = FunctionalDecoder(model.decoder)

    # dummy inputs (non-zero cache length required)
    mel = torch.randn(1, 80, 3000, dtype=torch.float32)
    audio = encoder(mel)

    n_text = model.dims.n_text_state
    n_audio = model.dims.n_audio_state
    n_self = len(decoder.keys_self_attn)
    n_cross = len(decoder.keys_cross_attn)

    cache_self = torch.zeros((n_self, 1, 1, n_text), dtype=torch.float32)
    cache_cross = torch.zeros((n_cross, 1, 1, n_audio), dtype=torch.float32)

    tokens = torch.zeros((1, 1), dtype=torch.long)

    # export encoder
    torch.onnx.export(
        encoder,
        (mel,),
        "encoder.onnx",
        input_names=["mel"],
        output_names=["audio"],
        dynamic_axes={
            "mel": {0: "batch", 2: "time"},
            "audio": {0: "batch", 1: "time"},
        },
        opset_version=OPSET,
    )

    # export decoder
    torch.onnx.export(
        decoder,
        (tokens, audio, cache_self, cache_cross),
        "decoder.onnx",
        input_names=[
            "tokens",
            "audio",
            "cache_self_attn",
            "cache_cross_attn",
        ],
        output_names=[
            "logits",
            "new_cache_self_attn",
            "new_cache_cross_attn",
        ],
        dynamic_axes={
            "tokens": {0: "batch", 1: "seq"},
            "audio": {0: "batch", 1: "time"},
            "cache_self_attn": {2: "cached_seq"},
            "cache_cross_attn": {2: "cached_time"},
            "new_cache_self_attn": {2: "cached_seq"},
            "new_cache_cross_attn": {2: "cached_time"},
        },
        opset_version=OPSET,
    )

    print("Export complete: encoder.onnx, decoder.onnx")


def _patch_decoder(model):
    """Replace attention with functional KV-cache aware version."""
    for block in model.decoder.blocks:
        block.attn.__class__ = FunctionalMultiHeadAttention
        block.cross_attn.__class__ = FunctionalMultiHeadAttention

        block.attn.n_ctx = model.dims.n_text_ctx
        block.cross_attn.n_ctx = model.dims.n_audio_ctx


class FunctionalDecoder(torch.nn.Module):
    """Decoder wrapper exposing KV cache as tensors."""

    def __init__(self, decoder):
        super().__init__()
        self.decoder = decoder

        self.keys_self_attn = []
        self.keys_cross_attn = []

        for block in decoder.blocks:
            self.keys_self_attn += (block.attn.key, block.attn.value)
            self.keys_cross_attn += (block.cross_attn.key, block.cross_attn.value)

    def forward(self, tokens, audio, cache_self, cache_cross):
        kv_cache = {
            **dict(zip(self.keys_self_attn, cache_self)),
            **dict(zip(self.keys_cross_attn, cache_cross)),
        }

        logits = self.decoder(tokens, audio, kv_cache=kv_cache)

        new_self = torch.cat(
            [kv_cache[k].unsqueeze(0) for k in self.keys_self_attn], dim=0
        )
        new_cross = torch.cat(
            [kv_cache[k].unsqueeze(0) for k in self.keys_cross_attn], dim=0
        )

        return logits, new_self, new_cross


class FunctionalMultiHeadAttention(MultiHeadAttention):
    """Attention with external KV cache support."""

    def forward(self, x, xa=None, mask=None, kv_cache=None):
        k, v = self._get_kv(x, xa, kv_cache)
        q = self.query(x)
        wv, qk = self.qkv_attention(q, k, v, mask)
        return self.out(wv), qk
        

    def _get_kv(self, x, xa, kv_cache):
        if kv_cache is None:
            src = xa if xa is not None else x
            return self.key(src), self.value(src)

        # cross-attention: compute once
        if xa is not None:
            if kv_cache[self.key].shape[1] <= 1:
                kv_cache[self.key] = self.key(xa).detach()
                kv_cache[self.value] = self.value(xa).detach()
            return kv_cache[self.key], kv_cache[self.value]

        # self-attention: incremental append
        key = torch.cat([kv_cache[self.key], self.key(x).detach()], dim=1)
        value = torch.cat([kv_cache[self.value], self.value(x).detach()], dim=1)

        kv_cache[self.key] = key[:, -self.n_ctx :, :]
        kv_cache[self.value] = value[:, -self.n_ctx :, :]

        return kv_cache[self.key], kv_cache[self.value]


def clear_existing_exports():
    for filename in ["encoder.onnx", "encoder.onnx.data", "decoder.onnx", "decoder.onnx.data"]:
        if os.path.exists(filename):
            os.remove(filename)


if __name__ == "__main__":
    
    clear_existing_exports()

    main()


/tmp/ipykernel_53297/359843868.py:36: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0207 18:56:00.714000 53297 torch/onnx/_internal/exporter/_compat.py:125] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W0207 18:56:01.292000 53297 torch/onnx/_internal/exporter/_registration.py:110] torchvision is not installed. Skipping torchvision::nms
W0207 18:56:01.294000 53297 torch/onnx/_internal/exporter/_registration.py:110] torchvi

[torch.onnx] Obtain model graph for `AudioEncoder([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `AudioEncoder([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


/usr/lib/python3.12/copyreg.py:99: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)
The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 17).


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 16 of general pattern rewrite rules.


/tmp/ipykernel_53297/359843868.py:50: UserWarning: Exporting a model while it is in training mode. Please ensure that this is intended, as it may lead to different behavior during inference. Calling model.eval() before export is recommended.
  torch.onnx.export(
/tmp/ipykernel_53297/359843868.py:50: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0207 18:56:06.836000 53297 torch/onnx/_internal/exporter/_compat.py:125] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_versio

[torch.onnx] Obtain model graph for `FunctionalDecoder([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `FunctionalDecoder([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


/usr/lib/python3.12/copyreg.py:99: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...


/home/pearlrubymv/reauthor-whisper-asr-android-runtime/android-dev/lib/python3.12/site-packages/torch/onnx/_internal/exporter/_onnx_program.py:460: UserWarning: # The axis name: batch will not be used, since it shares the same shape constraints with another axis: batch.
  rename_mapping = _dynamic_shapes.create_rename_mapping(
The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 17).


[torch.onnx] Translate the graph into ONNX... ✅
Applied 47 of general pattern rewrite rules.
Export complete: encoder.onnx, decoder.onnx


In [6]:
from gtts import gTTS

text = "Hello, this is a Whisper ONNX test."
tts = gTTS(text=text, lang="en")

tts.save("sample.mp3")


In [4]:
import whisper
import whisper.model
whisper.model.MultiHeadAttention.use_sdpa = False

model = whisper.load_model("tiny")

result = model.transcribe("sample.mp3")


print(result["text"])


 Hello, this is a whisper ONNX test.


In [20]:
import numpy as np
import torch
import whisper
import onnxruntime as ort

# load tokenizer
tokenizer = whisper.tokenizer.get_tokenizer(True)

# load audio
audio = whisper.load_audio("sample.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).unsqueeze(0).numpy()

# load ONNX sessions
enc = ort.InferenceSession("encoder.onnx")
dec = ort.InferenceSession("decoder.onnx")

# run encoder
audio_features = enc.run(None, {"mel": mel})[0]

# init KV cache (tiny.en = 4 layers × key,value = 8)
cache_self = np.zeros((8, 1, 1, 384), dtype=np.float32)
cache_cross = np.zeros((8, 1, 1, 384), dtype=np.float32)

# correct start tokens
tokens = [[tokenizer.sot, tokenizer.no_timestamps]]

# first decoder run
logits, cache_self, cache_cross = dec.run(
    None,
    {
        "tokens": np.array(tokens, dtype=np.int64),
        "audio": audio_features,
        "cache_self_attn": cache_self,
        "cache_cross_attn": cache_cross,
    },
)

# incremental decoding
for _ in range(100):
    next_token = logits[:, -1, :].argmax(axis=-1).astype(np.int64).reshape(1,1)

    if next_token[0,0] == tokenizer.eot:
        break

    tokens[0].append(int(next_token[0,0]))

    logits, cache_self, cache_cross = dec.run(
        None,
        {
            "tokens": next_token,
            "audio": audio_features,
            "cache_self_attn": cache_self,
            "cache_cross_attn": cache_cross,
        },
    )

# decode text
text = tokenizer.decode(tokens[0])
print(text)


<|startoftranscript|><|notimestamps|>atatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatatat
